In [1]:
import sqlite3
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
connection = sqlite3.connect('my_database2.db')
%load_ext sql
%sql sqlite:///my_database2.db

In [ ]:
# 1) Показать какой поставщик поставил каждый товар на склад (INNER JOIN).

In [7]:
%%sql
SELECT Product.name as product, Provider.name as provider FROM Store
INNER JOIN Product ON ID_Product = Product.id
INNER JOIN Provider ON ID_Provider = Provider.id

 * sqlite:///my_database2.db
Done.


product,provider
Футбольный мяч,Компания A
Футбольный мяч,Компания B
Ноутбук,Компания A
Ноутбук,Компания C
Зонтик,Компания B
Фотоаппарат,Компания C
Фотоаппарат,Компания A
Фотоаппарат,Компания A
Сумка,Компания A
Наушники,Компания B


In [ ]:
# 2) Вывести список товаров, которыми торгует фирма, и их поставщиков вне
# зависимости от наличия поставок (LEFT JOIN).

In [15]:
%%sql
SELECT Product.name as product, Provider.name as provider, quantity FROM Product
LEFT JOIN Store ON ID_Product = Product.id
LEFT JOIN Provider ON ID_Provider = Provider.id

 * sqlite:///my_database2.db
Done.


product,provider,quantity
Футбольный мяч,Компания A,100
Футбольный мяч,Компания B,150
Ноутбук,Компания A,50
Ноутбук,Компания C,70
Зонтик,Компания B,200
Фотоаппарат,Компания A,40
Фотоаппарат,Компания A,200
Фотоаппарат,Компания C,30
Сумка,Компания A,0
Наушники,Компания B,100


In [ ]:
# 3) Вывести информацию о покупаемых со склада товарах и их покупателях, включая
# товары, отсутствующие в списке реализованных товаров (RIGHT JOIN).

In [61]:
# %%sql
# SELECT 
#     Product.name, 
#     COALESCE(Client.name, "Не определен") as ClientName, 
#     COALESCE(Deal.quantity, "Нет продаж") as quantity FROM Store
# LEFT JOIN Deal ON Deal.ID_Store = Store.id
# LEFT JOIN Client ON ID_Client = Client.id
# LEFT JOIN Product ON ID_Product = Product.id

In [62]:
%%sql
SELECT 
    Product.name, 
    COALESCE(Client.name, "Не определен") as clientName,
    COALESCE(Deal.quantity, "Нет продаж") as quantity
FROM Deal
RIGHT JOIN Store ON ID_Store = Store.id
INNER JOIN Product ON ID_Product = Product.id
LEFT JOIN Client ON ID_Client = Client.id

 * sqlite:///my_database2.db
Done.


name,clientName,quantity
Футбольный мяч,Иванов Петр,5
Футбольный мяч,Смирнова Елена,10
Футбольный мяч,Козлов Александр,3
Ноутбук,Попова Марина,8
Футбольный мяч,Николаев Андрей,6
Ноутбук,Сергеева Ольга,4
Футбольный мяч,Петров Дмитрий,7
Футбольный мяч,Кузнецова Анна,7
Ноутбук,Морозова Наталья,9
Футбольный мяч,Морозова Наталья,11


In [ ]:
# 4) Вывести список поставщиков, которые хотя бы раз осуществляли поставку на склад
# (полусоединение).

In [66]:
%%sql
SELECT DISTINCT Provider.name FROM Store
INNER JOIN Provider ON ID_Provider = Provider.id

 * sqlite:///my_database2.db
Done.


name
Компания A
Компания B
Компания C


In [70]:
%%sql
SELECT DISTINCT Provider.name FROM Store
LEFT JOIN Provider ON ID_Provider = Provider.id

 * sqlite:///my_database2.db
Done.


name
Компания A
Компания B
Компания C


In [76]:
%%sql
SELECT DISTINCT Provider.name FROM Provider
RIGHT JOIN Store ON ID_Provider = Provider.id

 * sqlite:///my_database2.db
Done.


name
Компания A
Компания B
Компания C


In [88]:
%%sql
SELECT *
FROM Provider
WHERE EXISTS (
    SELECT 1
    FROM Store
    WHERE Provider.id = Store.ID_Provider
);

 * sqlite:///my_database2.db
Done.


id,name
1,Компания A
2,Компания B
3,Компания C


In [ ]:
# 5)Выведите список сотрудников с указанием их прямых начальников
# (самосоединение). Для главного начальника в столбец «Начальник» вывести
# «не определен».

In [103]:
%%sql
SELECT 
    e1.name, 
    COALESCE(e2.name, "Не определен") as manager
    FROM Employee e1
LEFT JOIN Employee e2 ON e2.id = e1.ID_Manager

 * sqlite:///my_database2.db
Done.


name,manager
Иванов Павел,Не определен
Петров Алексей,Иванов Павел
Сидорова Елена,Иванов Павел
Козлова Анастасия,Петров Алексей
Смирнов Дмитрий,Петров Алексей
Попова Ольга,Петров Алексей
Николаев Игорь,Сидорова Елена
Сергеева Марина,Сидорова Елена
Петров Денис,Сидорова Елена
